# Score DIstribution of Tasks (Rubric)
### Task 1: 20%
### Task 2: 40%
### Task 3: 40 %

#Task 1: Run and understand the tutorial
# Smoothing techniques commonly used in NLP

In this notebook, I will introduce several smoothing techniques commonly used in NLP or machine learning algorithms. They are:
- Laplacian (add-one) Smoothing
- Lidstone (add-k) Smoothing
- Absolute Discounting
- Katz Backoff
- Kneser-Ney Smoothing
- Interpolation

Before starting to implement these smoothing methods, we first need to build a *N*-gram language model, and then applying different smoothing methods to this language model, evaluating the results between these smoothing techniques. In this notebook, I will build a bigram language model.

Now, let's define some notations used in the following programs. In this notebook, **token** is the number of words in a document or a sentence, **vocab** is the number of different type of words in a document or sentence. For example, in the following sentence, there are 10 tokens and 8 vocabs (because "I" and "like" occur two times).  
"I like natural language processing and I like machine learning."

## Bigram Language Model

In [1]:
from collections import defaultdict
from collections import Counter
from numpy.random import choice 
from tqdm import tqdm

class Bigram():
    def __init__(self):
        self.bigram_counts = defaultdict(Counter)
        self.unigram_counts = Counter()
        self.context = defaultdict(Counter)
        self.start_count = 0
        self.token_count = 0
        self.vocab_count = 0
    
    def convert_sentence(self, sentence):
        return ["<s>"] + [w.lower() for w in sentence] + ["</s>"]
    
    def get_counts(self, sentences):
        # collect unigram counts
        for sentence in sentences:
            sentence = self.convert_sentence(sentence)
            for word in sentence[1:]:  # from 1, because we don't need the <s> token
                self.unigram_counts[word] += 1
            self.start_count += 1
            
        # collect bigram counts
        for sentence in sentences:
            sentence = self.convert_sentence(sentence)
            bigram_list = zip(sentence[:-1], sentence[1:])
            for bigram in bigram_list:
                self.bigram_counts[bigram[0]][bigram[1]] += 1
                self.context[bigram[1]][bigram[0]] += 1
        self.token_count = sum(self.unigram_counts.values())
        self.vocab_count = len(self.unigram_counts.keys())
        
    def generate_sentence(self):
        current_word = "<s>"
        sentence = [current_word]
        while current_word != "</s>":
            prev_word = current_word
            prev_word_counts = self.bigram_counts[prev_word]
            # obtain bigram probability distribution given the previous word
            bigram_probs = []
            total_counts = float(sum(prev_word_counts.values()))
            for word in prev_word_counts:
                bigram_probs.append(prev_word_counts[word] / total_counts)
            # sample the next word
            current_word = choice(list(prev_word_counts.keys()), p=bigram_probs)
            sentence.append(current_word)
            
        sentence = " ".join(sentence[1:-1])
        return sentence

Now we have finished building our bigram language model without any smoothing. Let's try to generate some sentences using the Penn Treebank corpora as training data.

In [2]:
import nltk
from nltk.corpus import brown
# nltk.download('brown')

bigram = Bigram()
bigram.get_counts(brown.sents())
for i in range(1,6):
    print("Sentence %d" % i)
    print(bigram.generate_sentence())

Sentence 1
you have to my father seemed nothing has settled her fear .
Sentence 2
russ wiped some cases .
Sentence 3
by a more familiar with his guns and move .
Sentence 4
at the youngest clerk was the difference is astarte , aimed wholly in the mount vernon reports .
Sentence 5
figures i've never be memorized .


The output for our sample sentence looks reasonable, Now, let's use perplexity to evaluate different smoothing techniques at the level of the corpus. For this, we'll divide Brown corpus up randomly into a training set and a test set based on an 80/20 split. The perplexity can be calculated as follow:

$PP(W) = \sqrt[m]{\frac{1}{P(W)}}$

$\log{PP(W)} = -\frac{1}{m} \log{P(W)}$

In [3]:
import math
from random import shuffle

def split_train_test():
    sents = list(brown.sents())
    shuffle(sents)
    cutoff = int(0.8*len(sents))
    training_set = sents[:cutoff]
    test_set = [[word.lower() for word in sent] for sent in sents[cutoff:]]
    return training_set, test_set

def calculate_perplexity(sentences, bigram, smoothing_function, parameter):
    total_log_prob = 0
    test_token_count = 0
    for sentence in tqdm(sentences):
        test_token_count += len(sentence) + 1 # have to consider the end token
        total_log_prob += smoothing_function(sentence, bigram, parameter)
    return math.exp(-total_log_prob / test_token_count)

training_set, test_set = split_train_test()

Until Now, we can evaluate the quality of different smoothing methods via calculating perplexity of test set. Now let's start to learn these smoothing techniques. For better understanding, here we use a sample example to explain these smoothing methods. Supposing we have 7 vocabs and their counts are as follows: **(Note this is a simplified example which is more like a unigram model)**

vocabs | counts | unsmoothed probability
:-: | :-: | :-: 
impropriety | 8 | 0.4 | 
offense | 5 | 0.25 | 
damage | 4 | 0.2 | 
deficiencies | 2 | 0.1 | 
outbreak | 1 | 0.05 | 
infirmity | 0 | 0 | 
cephalopods | 0 | 0 | 
**total** | **20** | **1.0** | 

A bigram model without any smoothing can be formulated as follow: 
$$ P(w_{i}|w_{i-1}) = \frac{C(w_{i-1}, w_{i})}{C(w_{i-1})} $$

## Laplacian (add-one) Smoothing

Laplacian (add-one) smoothing: 

$$ P_{add-1}(w_{i}|w_{i-1}) = \frac{C(w_{i-1}, w_{i}) + 1}{C(w_{i-1}) + |V|}$$

**Core idea**: Pretend that we have seen each vocab at least once.

vocabs | counts | unsmoothed probability | laplacian (add-one) smoothing
:-: | :-: | :-: | :-: 
impropriety | 8 | 0.4 | (8+1)/(20+7)= 0.333
offense | 5 | 0.25 | (5+1)/(20+7)= 0.222
damage | 4 | 0.2 | (4+1)/(20+7)= 0.186
deficiencies | 2 | 0.1 | (2+1)/(20+7)= 0.111
outbreak | 1 | 0.05 | (1+1)/(20+7)= 0.074
infirmity | 0 | 0 | (0+1)/(20+7)= 0.037
cephalopods | 0 | 0 | (0+1)/(20+7)= 0.037
**total** | **20** | **1.0** | **1.0**

In [4]:
def laplacian_smoothing(sentence, bigram, parameter):
    sentence = bigram.convert_sentence(sentence)
    bigram_list = zip(sentence[:-1], sentence[1:])
    prob = 0
    for prev_word, word in bigram_list:
        sm_bigram_counts = bigram.bigram_counts[prev_word][word] + 1
        if prev_word == "<s>": sm_unigram_counts = bigram.start_count
        else: sm_unigram_counts = bigram.unigram_counts[prev_word] + len(bigram.unigram_counts)
        prob += math.log(sm_bigram_counts / sm_unigram_counts)
    return prob

bigram_laplacian_smoothing = Bigram()
bigram_laplacian_smoothing.get_counts(training_set)
plex_laplacian_smoothing = calculate_perplexity(test_set, bigram_laplacian_smoothing, laplacian_smoothing, None)
print(plex_laplacian_smoothing)

100%|█████████████████████████████████████████████████████████████████████████| 11468/11468 [00:00<00:00, 27625.56it/s]

3486.254398478114


In [12]:
text = [['Meme-based', 'cryptocurrency', 'shiba', 'inu', 'soared', 'more', 'than', '45%', 'over', 'the', 'past', '24', 'hours,', 'muscling', 
         'into', 'the', 'top-10', 'largest', 'digital', 'tokens', 'by', 'market', 'capitalization.'], 
        ['Shiba', 'inu', 'is', 'a', 'spinoff', 'of', 'dogecoin,', 'itself', 'born', 'as', 'a', 'satire', 'of', 'a', 'cryptocurrency', 'frenzy', 
         'in', '2013,', 'and', 'has', 'barely', 'any', 'practical', 'use.']]
bigram_laplacian_smoothing = Bigram()
bigram_laplacian_smoothing.get_counts(text)

def test(sentence, bigram):
    sentence = bigram.convert_sentence(sentence)
    bigram_list = zip(sentence[:-1], sentence[1:])
    prob = 0
    for prev_word, word in bigram_list:
        sm_bigram_counts = bigram.bigram_counts[prev_word][word] + 1
        print(f'first sm_bigram_counts: {sm_bigram_counts}')
        if prev_word == "<s>": 
            sm_unigram_counts = bigram.start_count
        else: 
            sm_unigram_counts = bigram.unigram_counts[prev_word] + len(bigram.unigram_counts)
            print(f'sm_unigram_counts {sm_unigram_counts}')
        prob += math.log(sm_bigram_counts / sm_unigram_counts)
        print(f'prob: {prob}')
    return prob

test(text, bigram_laplacian_smoothing)

AttributeError: 'list' object has no attribute 'lower'

In [11]:
unigram_laplacian_smoothing = Bigram()
unigram_laplacian_smoothing.get_counts(text)

print('Bigram counts')
print(unigram_laplacian_smoothing.bigram_counts)
print()
print('Unigram counts')
print(unigram_laplacian_smoothing.unigram_counts)
print()
print('Context')
print(unigram_laplacian_smoothing.context)
print()
print('Start count')
print(unigram_laplacian_smoothing.start_count)
print()
print('Token count')
print(unigram_laplacian_smoothing.token_count)
print()
print('Vocab count')
print(unigram_laplacian_smoothing.vocab_count)
print()

Bigram counts
defaultdict(<class 'collections.Counter'>, {'<s>': Counter({'meme-based': 1, 'shiba': 1}), 'meme-based': Counter({'cryptocurrency': 1}), 'cryptocurrency': Counter({'shiba': 1, 'frenzy': 1}), 'shiba': Counter({'inu': 2}), 'inu': Counter({'soared': 1, 'is': 1}), 'soared': Counter({'more': 1}), 'more': Counter({'than': 1}), 'than': Counter({'45%': 1}), '45%': Counter({'over': 1}), 'over': Counter({'the': 1}), 'the': Counter({'past': 1, 'top-10': 1}), 'past': Counter({'24': 1}), '24': Counter({'hours,': 1}), 'hours,': Counter({'muscling': 1}), 'muscling': Counter({'into': 1}), 'into': Counter({'the': 1}), 'top-10': Counter({'largest': 1}), 'largest': Counter({'digital': 1}), 'digital': Counter({'tokens': 1}), 'tokens': Counter({'by': 1}), 'by': Counter({'market': 1}), 'market': Counter({'capitalization.': 1}), 'capitalization.': Counter({'</s>': 1}), 'is': Counter({'a': 1}), 'a': Counter({'spinoff': 1, 'satire': 1, 'cryptocurrency': 1}), 'spinoff': Counter({'of': 1}), 'of': C

In [ ]:
def unilaplacian_smoothing(sentence, bigram, parameter):
    sentence = bigram.convert_sentence(sentence)
    unigram_list = sentence
    prob = 0
    for word in unigram_list:
        sm_unigram_counts = bigram.unigram_counts[word] + 1
        if 
            word == "<s>": sm_unigram_counts = bigram.start_count
        else: 
#             sm_unigram_counts = bigram.unigram_counts[word] + len(bigram.unigram_counts)
            
        prob += math.log(sm_bigram_counts / sm_unigram_counts)
    return prob

## Lidstone (add-k) Smoothing

Lidstone (add-k) smoothing: 

$$ P_{add-k}(w_{i}|w_{i-1}) = \frac{C(w_{i-1}, w_{i}) + k}{C(w_{i-1}) + k|V|}$$

**Core idea**: Sometimes adding one is too much, instead, we add k (usually k < 1).

vocabs | counts | unsmoothed probability | lidstone (add-k) smoothing (k=0.05)
:-: | :-: | :-: | :-: 
impropriety | 8 | 0.4 | (8+0.5)/(20+7*0.5)= 0.363
offense | 5 | 0.25 | (5+0.5)/(20+7*0.5)= 0.234
damage | 4 | 0.2 | (4+0.5)/(20+7*0.5)= 0.191
deficiencies | 2 | 0.1 | (2+0.5)/(20+7*0.5)= 0.106
outbreak | 1 | 0.05 | (1+0.5)/(20+7*0.5)= 0.064
infirmity | 0 | 0 | (0+0.5)/(20+7*0.5)= 0.021
cephalopods | 0 | 0 | (0+0.5)/(20+7*0.5)= 0.021
**total** | **20** | **1.0** | **1.0**

In [ ]:
def lidstone_smoothing(sentence, bigram, k):
    sentence = bigram.convert_sentence(sentence)
    bigram_list = zip(sentence[:-1], sentence[1:])
    prob = 0
    for prev_word, word in bigram_list:
        sm_bigram_counts = bigram.bigram_counts[prev_word][word] + k
        if prev_word == "<s>": sm_unigram_counts = bigram.start_count
        else: sm_unigram_counts = bigram.unigram_counts[prev_word] + k*len(bigram.unigram_counts)
        prob += math.log(sm_bigram_counts / sm_unigram_counts)
    return prob

bigram_lidstone_smoothing = Bigram()
bigram_lidstone_smoothing.get_counts(training_set)
plex_lidstone_smoothing = calculate_perplexity(test_set, bigram_lidstone_smoothing, lidstone_smoothing, 0.05)
print(plex_lidstone_smoothing)

## Absolute Discounting

Absolute discounting:

$$ P_{absolute-discounting}(w_{i}|w_{i-1})=\left\{
\begin{aligned}
\frac{C(w_{i-1}, w_{i}) - D}{C(w_{i-1})}, if \quad C(w_{i-1}, w_{i}) > 0 \\
\alpha(w_{i-1}) / \sum\nolimits_{w_{j}:C(w_{i-1}, w_{j})=0}, otherwise
\end{aligned}
\right.
$$

**Core idea**: 'Borrows' a fixed probability mass from observed n-gram counts and redistributes it to unseen n-grams.

$\alpha(w_{i-1})$ is the amount of probability mass that has been discounted for context $w_{i-1}$, in this example, its valuse is (0.1*5)/20.

$\sum\nolimits_{w_{j}:C(w_{i-1}, w_{j})=0}$ is the count of $C(w_{i-1}, w_{j})=0$, here it is 2.

vocabs | counts | unsmoothed probability | absolute discounting (d=0.1) | effective counts
:-: | :-: | :-: | :-: | :-: 
impropriety | 8 | 0.4 | (8-0.1)/20=0.395 | 7.9
offense | 5 | 0.25 | (5-0.1)/20=0.245 | 4.9
damage | 4 | 0.2 | (4-0.1)/20=0.195 | 3.9
deficiencies | 2 | 0.1 | (2-0.1)/20=0.095 | 1.9
outbreak | 1 | 0.05 | (1-0.1)/20=0.045 | 0.9
infirmity | 0 | 0 | (0+0.5)/20/2=0.0125 | 0.25
cephalopods | 0 | 0 | (0+0.5)/20/2=0.0125 | 0.25
**total** | **20** | **1.0** | **1.0** | **20**

In [ ]:
def absolute_discounting(sentence, bigram, d):
    sentence = bigram.convert_sentence(sentence)
    bigram_list = zip(sentence[:-1], sentence[1:])
    prob = 0

    for prev_word, word in bigram_list:
        sm_bigram_counts = bigram.bigram_counts[prev_word][word]
        if prev_word == "<s>": sm_unigram_counts = bigram.start_count
        else: sm_unigram_counts = bigram.unigram_counts[prev_word]
        if sm_unigram_counts == 0: 
            prob += math.log((1 / float(bigram.vocab_count)) * 0.01)
            continue
        if sm_bigram_counts != 0: 
            sm_bigram_counts = sm_bigram_counts - d
        else: 
            alpha_prev_word = len(bigram.bigram_counts[prev_word].keys())
            # count how many vocabs do not appear after pre_word
            prev_word_discounting = bigram.vocab_count - alpha_prev_word
            sm_bigram_counts = alpha_prev_word * d / prev_word_discounting
        prob += math.log(sm_bigram_counts / sm_unigram_counts)
    return prob

bigram_absolute_discounting = Bigram()
bigram_absolute_discounting.get_counts(training_set)
plex_absolute_discounting = calculate_perplexity(test_set, bigram_absolute_discounting, absolute_discounting, 0.1)
print(plex_absolute_discounting)

## Katz Backoff

Katz Backoff:

$$ P_{backoff}(w_{i}|w_{i-1})=\left\{
\begin{aligned}
\frac{C(w_{i-1}, w_{i}) - D}{C(w_{i-1})}, if \quad C(w_{i-1}, w_{i}) > 0 \\
\alpha(w_{i-1}) \times \frac{P(w_{j})}{\sum\nolimits_{w_{j}:C(w_{i-1}, w_{j})=0}{P(w_{j})}}, otherwise
\end{aligned}
\right.
$$

**Core idea**: Absolute discounting redistributes the probability mass **equally** for all unseen n-grams while Backoff redistributes the mass based on a lower order model (e.g. unigram).

$\alpha(w_{i-1})$ is also the amount of probability mass that has been discounted for context $w_{i-1}$, in this example, its valuse is (0.1*5)/20.

$P(w_{i})$ is the unigram probability for $w_{i}$. Suppose here $P(infirmity) = 0.002$, $P(cephalopods) = 0.008$.

vocabs | counts | unsmoothed probability | backoff | effective counts
:-: | :-: | :-: | :-: | :-: 
impropriety | 8 | 0.4 | (8-0.1)/20=0.395 | 7.9
offense | 5 | 0.25 | (5-0.1)/20=0.245 | 4.9
damage | 4 | 0.2 | (4-0.1)/20=0.195 | 3.9
deficiencies | 2 | 0.1 | (2-0.1)/20=0.095 | 1.9
outbreak | 1 | 0.05 | (1-0.1)/20=0.045 | 0.9
infirmity | 0 | 0 | (0+0.5)/20*0.002/(0.002+0.008)=0.0005 | 0.1
cephalopods | 0 | 0 | (0+0.5)/20*0.008/(0.002+0.008)=0.02 | 0.4
**total** | **20** | **1.0** | **1.0** | **20**

In [ ]:
def backoff(sentence, bigram, d):
    sentence = bigram.convert_sentence(sentence)
    bigram_list = zip(sentence[:-1], sentence[1:])
    prob = 0

    for prev_word, word in bigram_list:
        sm_bigram_counts = bigram.bigram_counts[prev_word][word]
        if prev_word == "<s>": sm_unigram_counts = bigram.start_count
        else: sm_unigram_counts = bigram.unigram_counts[prev_word]
        if sm_unigram_counts == 0: 
            prob += math.log((1 / float(bigram.vocab_count)) * 0.01)
            continue
        if sm_bigram_counts != 0: 
            sm_bigram_counts = sm_bigram_counts - d
        else: 
            alpha_prev_word = len(bigram.bigram_counts[prev_word].keys())
            # sum unigram counts of word j which do not appear after pre_word
            unseen_unigram_sum = 0
            for vocab in bigram.unigram_counts.keys():
                if vocab not in bigram.bigram_counts[prev_word].keys():
                    unseen_unigram_sum += bigram.unigram_counts[vocab]
            unseen_unigram = bigram.unigram_counts[word] / unseen_unigram_sum
            if unseen_unigram == 0: unseen_unigram = 1 / float(bigram.vocab_count - alpha_prev_word)
            sm_bigram_counts = alpha_prev_word * d * unseen_unigram
        prob += math.log(sm_bigram_counts / sm_unigram_counts)
    return prob

bigram_backoff = Bigram()
bigram_backoff.get_counts(training_set)
plex_backoff = calculate_perplexity(test_set, bigram_backoff, backoff, 0.1)
print(plex_backoff)

## Kneser-Ney Smoothing

Kneser-Ney Smoothing:

$$ P_{kneser-ney-smoothing}(w_{i}|w_{i-1})=\left\{
\begin{aligned}
\frac{C(w_{i-1}, w_{i}) - D}{C(w_{i-1})}, if \quad C(w_{i-1}, w_{i}) > 0 \\
\alpha(w_{i-1})P_{cont}(w_{i}), otherwise
\end{aligned}
\right.\\
where \quad
P_{cont}(w_{i}) = \frac{|\{w_{i-1}:C(w_{i-1}, w_{i}) > 0\}|}{{\sum_{w_{i}}{|\{w_{i-1}:C(w_{i-1}, w_{i}) > 0\}|}}}
$$

**Core idea**: Redistribute probability mass based on how many number of different contexts word w has appeared in.

$\alpha(w_{i-1})$ is also the amount of probability mass that has been discounted for context $w_{i-1}$, in this example, its valuse is (0.1*5)/20.  
Suppose we have the following phrases in the corpus: {A infirmity, B infirmity, C infirmity, D infirmity, A cephalopods}, then  
$|\{w_{i-1}:C(w_{i-1}, w_{i}) > 0\}|$ for $w_{i}$ = infirmity is 4, $P_{cont}(w_{i}=infirmity)$ = 4/(4+1)= 0.8.  
$|\{w_{i-1}:C(w_{i-1}, w_{i}) > 0\}|$ for $w_{i}$ = cephalopods is 1, $P_{cont}(w_{i}=cephalopods)$ = 1/(4+1)= 0.2

vocabs | counts | unsmoothed probability | kneser-ney smoothing | effective counts
:-: | :-: | :-: | :-: | :-: 
impropriety | 8 | 0.4 | (8-0.1)/20=0.395 | 7.9
offense | 5 | 0.25 | (5-0.1)/20=0.245 | 4.9
damage | 4 | 0.2 | (4-0.1)/20=0.195 | 3.9
deficiencies | 2 | 0.1 | (2-0.1)/20=0.095 | 1.9
outbreak | 1 | 0.05 | (1-0.1)/20=0.045 | 0.9
infirmity | 0 | 0 | (0+0.5)/20*4/(4+1)=0.02 | 0.4
cephalopods | 0 | 0 | (0+0.5)/20*1/(4+1)=0.005 | 0.1
**total** | **20** | **1.0** | **1.0** | **20**

In [ ]:
def kneser_ney_smoothing(sentence, bigram, d):
    sentence = bigram.convert_sentence(sentence)
    bigram_list = zip(sentence[:-1], sentence[1:])
    prob = 0

    for prev_word, word in bigram_list:
        sm_bigram_counts = bigram.bigram_counts[prev_word][word]
        if prev_word == "<s>": sm_unigram_counts = bigram.start_count
        else: sm_unigram_counts = bigram.unigram_counts[prev_word]
        if sm_unigram_counts == 0: 
            prob += math.log((1 / float(bigram.vocab_count)) * 0.01)
            continue
        if sm_bigram_counts != 0: 
            sm_bigram_counts = sm_bigram_counts - d
        else: 
            # statistic how many tokens not occureed after pre_word
            alpha_prev_word = len(bigram.bigram_counts[prev_word].keys())
            
            context_sum = 0
            for vocab in bigram.unigram_counts.keys():
                if vocab not in bigram.bigram_counts[prev_word].keys():
                    context_sum += len(bigram.context[vocab].keys())
            p_cont = len(bigram.context[word].keys()) / context_sum
            if p_cont == 0: p_cont = 1 / float(bigram.vocab_count - alpha_prev_word)
            sm_bigram_counts = alpha_prev_word * d * p_cont
        prob += math.log(sm_bigram_counts / sm_unigram_counts)
    return prob

bigram_kneser_ney_smoothing = Bigram()
bigram_kneser_ney_smoothing.get_counts(training_set)
plex_kneser_ney_smoothing = calculate_perplexity(test_set, bigram_kneser_ney_smoothing, kneser_ney_smoothing, 0.1)
print(plex_kneser_ney_smoothing)

## Interpolation

Interpolation:

$$ 
\begin{aligned}
P_{interpolation}(w_{i}|w_{i-1}, w_{i-2})&=\lambda_{3}P_{3}(w_{i}|w_{i-1}, w_{i-2}) \\
&+\lambda_{2}P_{2}(w_{i}|w_{i-1})\\
&+\lambda_{1}P_{1}(w_{i})\\
where \quad
\sum_{i}{\lambda_{i}} = 1
\end{aligned}
$$

**Core idea**: Combine different order n-gram models.

In [ ]:
def interpolation(sentence, bigram, lambdas):
    bigram_lambda = lambdas[0]
    unigram_lambda = lambdas[1]
    zerogram_lambda = 1 - lambdas[0] - lambdas[1]
    
    sentence = bigram.convert_sentence(sentence)
    bigram_list = zip(sentence[:-1], sentence[1:])
    prob = 0
    
    for prev_word, word in bigram_list:
        # bigram probability
        sm_bigram_counts = bigram.bigram_counts[prev_word][word]
        if sm_bigram_counts == 0: interp_bigram_counts = 0
        else:
            if prev_word == "<s>": u_counts = bigram.start_count
            else: u_counts = bigram.unigram_counts[prev_word]
            interp_bigram_counts = sm_bigram_counts / float(u_counts) * bigram_lambda

        # unigram probability
        interp_unigram_counts = (bigram.unigram_counts[word] / bigram.token_count) * unigram_lambda

        # "zerogram" probability: this is to account for out-of-vocabulary words, this is just 1 / |V|
        vocab_size = len(bigram.unigram_counts)
        interp_zerogram_counts = (1 / float(vocab_size)) * zerogram_lambda
    
        prob += math.log(interp_bigram_counts + interp_unigram_counts + interp_zerogram_counts)
    return prob

bigram_interpolation = Bigram()
bigram_interpolation.get_counts(training_set)
plex_interpolation = calculate_perplexity(test_set, bigram_interpolation, interpolation, (0.8, 0.19))
print(plex_interpolation)

Now we have finished our work, the following table shows the perplexity of different smoothing methods. We can learn that different smoothing techniques may greatly affect the quality of language models. 

smoothing techniques | perpleity on Brown test corpus
:-: | :-: 
Laplacian (add-one) Smoothing | 3508
Lidstone (add-k) Smoothing | 1188
Absolute Discounting | 1013
Katz Backoff | 588
Kneser-Ney Smoothing | 569
Interpolation | 436

The above implementations may not be optimal according to efficiency and memory, but it shows how different smoothing techniques work in a language model intuitively, so it may be a good tutorial for some beginners of NLP. If there are some mistakes in the code, welcome to point it out and I will correct it as soon as possible.

# Task 2
## Implement the above smoothing techinques for Unigram Model

In [ ]:
def unigram_laplacian_smoothing(sentence, bigram, parameter):
    sentence = bigram.convert_sentence(sentence)
    unigram_list = sentence
    prob = 0
    for prev_word, word in unigram_list:
        sm_unigram_counts = bigram.unigram_counts[prev_word][word] + 1
        if prev_word == "<s>": sm_unigram_counts = bigram.start_count
        else: sm_unigram_counts = bigram.unigram_counts[prev_word] + len(bigram.unigram_counts)
        prob += math.log(sm_bigram_counts / sm_unigram_counts)
    return prob

bigram_laplacian_smoothing = Bigram()
bigram_laplacian_smoothing.get_counts(training_set)
plex_laplacian_smoothing = calculate_perplexity(test_set, bigram_laplacian_smoothing, laplacian_smoothing, None)
print(plex_laplacian_smoothing)

# Task 3
## Implement the above smoothing techinques for Trigram Model